In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import generate_df as G
from IPython.display import display, HTML
from tabulate import tabulate
from unidecode import unidecode


In [2]:
spark = SparkSession.builder.appName("finance_data_processing")\
.config("spark.jars","/home/daiane/Downloads/postgresql-42.7.3.jar").getOrCreate()


24/07/22 15:00:30 WARN Utils: Your hostname, victor-Lenovo-ideapad-330-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.0.125 instead (on interface wlp2s0)
24/07/22 15:00:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/22 15:00:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
def cnpj_completo(cnpj_base_input):
    #adiciona "0001" e calcula dígitos verificadores, retorna cnpj com 14 dígitos
    pesos1 = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    pesos2 = [6, 5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]

    cnpj1 = cnpj_base_input + "0001"
    cnpj_base = cnpj1.zfill(12)

    #1° dígito verficador
    soma1 = sum(int(digito) * peso for digito, peso in zip(cnpj_base, pesos1))
    resto1 = soma1 % 11
    digito1 = 0 if resto1 < 2 else 11 - resto1

    #2° dígito verificador
    soma2 = sum(int(digito) * peso for digito, peso in zip(cnpj_base + str(digito1), pesos2))
    resto2 = soma2 % 11
    digito2 = 0 if resto2 < 2 else 11 - resto2

    cnpj = cnpj_base  + str(digito1) + str(digito2)
    return cnpj

cnpj_udf = F.udf(cnpj_completo, T.StringType())


In [4]:
df_demonstracao_result_ = G.table_to_df("finance_raw_data", "demonstracao_resultado_receita_lucro", spark)
df_demonstracao_result_.persist()

df_date_with_day = df_demonstracao_result_.withColumn("data_format_d", F.concat(F.lit("01/"), df_demonstracao_result_["data"]))

df_date_format = df_date_with_day.withColumn("data_format_d", F.to_date(df_date_with_day["data_format_d"], "dd/MM/yyyy"))
df_demonstracao_result = df_date_format.withColumn("cnpj_completo", cnpj_udf(df_date_format["codigo"]))

df_demonstracao_result = df_demonstracao_result.drop("data").filter(F.col("data_format_d").isNotNull())

df_demonstracao_result = df_demonstracao_result.withColumnRenamed("data_format_d", "data_trimestral")


df_demonstracao_result.persist()


t2 = df_demonstracao_result.sample(False, 0.1).limit(30)
# t = dfSample_demonstracao_result.withColumn("cnpj_completo", cnpj_udf(dfSample_demonstracao_result["codigo"]))

# t2 = t.select("instituicao", "codigo", "cnpj_completo")
table = tabulate(t2.collect(), headers=t2.columns, tablefmt='html')

# t2.printSchema()
display(HTML(table))
# print(dfSample_demonstracao_result.count())




24/07/22 15:01:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/07/22 15:01:58 WARN TaskSetManager: Stage 0 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:02:07 WARN TaskSetManager: Stage 1 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:02:18 WARN TaskSetManager: Stage 2 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


instituicao,codigo,conglomerado,codigo_conglomerado_financeiro,codigo_conglomerado_prudencial,tcb,tc,ti,cidade,uf,rendas_operacoes_credito_a1,rendas_operacoes_arrendamento_mercantil_a2,rendas_operacoes_com_tvm_a3,rendas_operacoes_instrumentos_financeiros_derivativos_a4,resultado_operacoes_cambio_a5,rendas_aplicacoes_compulsorias_a6,receitas_intermediacao_financeira_soma_a1_a2_a3_a4_a5_a6,despesas_captacao_b1,despesas_obrigacoes_por_emprestimos_e_repasses_b2,despesas_operacoes_arrendamento_mercantil_b3,resultado_operacoes_cambio_b4,resultado_de_provisao_para_creditos_dificil_liquidacao_b5,despesas__intermediacao_financeira_soma_b1_b2_b3_b4_b5,resultao_intermediacao_financeira_c_soma_a_b,rendas_prestacao_servico_d1,rendas_tarifas_bancarias_d2,despesas_de_pessoal_d3,despesas_administrativas_d4,despesas_tributarias_d5,resultado_de_participacoes_d6,outras_receitas_operacionais_d7,outras_receitas_operacionais_d8,soma_d1_d2_d3_d4_d5_d6_d7_d8,resultado_operacional_soma_c_d,resultado_nao_operacional_f,g,imposto_renda_contribuicao_social_h,participacao_lucros_i,lucro_liquido_soma_g_h_i,juros_capital_social_cooperativas_k,data_trimestral,cnpj_completo
SAFRA LEASING S.A. ARRENDAMENTO MERCANTIL,62063177,SAFRA,10083,,n1,2,19,SAO PAULO,SP,5.856,677.285,899.635,-5.59,0,0,1.577.186,-834.522,-6.712,-648.661,0,19.919,-1.469.976,107.21,14,2,-3.282,-3.358,-3.982,0,2.49,-58.496,-66.611,40.599,-711,39.888,-29.037,0,10.851,0,2012-06-01,62063177000194
BANCO J. SAFRA S.A.,3017677,SAFRA,10083,,b1,2,8,SAO PAULO,SP,684.166,0,55.261,-69.543,0,22.664,692.549,-367.2,-43.938,0,0,-135.005,-546.143,146.406,72.266,28.732,-66.65,-57.067,-22.446,0,12.676,-23.982,-56.473,89.933,-710,89.223,-82.385,0,6.838,0,2012-06-01,03017677000120
BANCO PINE S.A.,62144175,PINE,50304,,b1,2,8,SAO PAULO,SP,202.385,0,284.576,21.199,114.53,0,622.689,-356.383,-89.744,0,0,-19.437,-465.564,157.125,24.406,2.313,-42.793,-37.06,-3.704,19.039,31.947,-13.195,-19.046,138.079,4.326,142.405,-21.358,-28.986,92.06,-30.008,2012-06-01,62144175000120
"PERNAMBUCANAS FINANCIADORA S.A. - CRÉDITO, FINANCIAMENTO E INVESTIMENTO",43180355,PERNAMBUCANAS,40699,,n1,2,14,SAO PAULO,SP,435.138,0,7.767,0,0,0,442.905,-21.897,0,0,-1,-167.483,-189.381,253.523,2.596,18.625,-28.108,-63.625,-987,-127,22.509,-57.997,-107.114,146.409,-12,146.397,-59.091,0,87.306,0,2012-06-01,43180355000112
BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL,92816560,,,,b4,1,5,PORTO ALEGRE,RS,358.891,0,55.307,0,0,0,414.199,0,-198.207,0,0,-96.797,-295.004,119.195,5.705,0,-56.901,-16.143,-7.212,0,88.065,-87,-73.487,45.707,499,46.206,-14.599,0,31.607,0,2012-06-01,92816560000137
COMPANHIA DE ARRENDAMENTO MERCANTIL RCI BRASIL,62307848,SANTANDER,30379,,n1,3,19,SAO PAULO,SP,722,398.586,2,0,0,0,399.311,-34.951,0,-333.647,0,-4.949,-373.547,25.763,0,15.917,0,-3.425,-6.575,0,4.737,-19.331,-8.677,17.086,51.004,68.09,-27.244,0,40.846,0,2012-06-01,62307848000115
BCV - BANCO DE CRÉDITO E VAREJO S/A,50585090,BMG,30290,,b1,2,8,BELO HORIZONTE,MG,192.838,0,60.949,17.184,520,0,271.491,-126.626,0,0,0,77.975,-48.650,222.841,114,250,-6.133,-73.109,-9.235,7.218,14.126,-52.808,-119.575,103.266,147,103.413,-38.244,-1.563,63.606,0,2012-06-01,50585090000106
ASSOCIACAO DE POUPANCA E EMPRESTIMO - POUPEX,655522,,,,n1,2,25,BRASILIA,DF,88.637,0,77.954,0,0,26.207,192.799,-114.479,0,0,0,-3.772,-118.251,74.548,0,1.672,-27.491,-18.81,-5.713,0,804,-9.077,-58.614,15.934,2.824,18.758,0,0,18.758,0,2012-06-01,00655522000121
BANCO INTERMEDIUM S/A,416968,,,,b1,2,8,BELO HORIZONTE,MG,94.892,0,19.536,0,0,0,114.428,-33.358,-2,0,0,-23.782,-57.142,57.285,1.256,992,-6.572,-47.735,-577,2.705,3.564,-2.714,-49.082,8.204,-478,7.725,-767,0,6.958,-3.335,2012-06-01,00416968000101
BANCO CIFRA S/A,62421979,BMG,30290,,b1,2,8,BELO HORIZONTE,MG,105.467,0,970,0,0,0,106.437,-29.766,0,0,0,-25.159,-54.926,51.511,365,89,-9.414,-40.632,-3.065,0,17.707,-15.724,-50.674,837,120,957,-363,0,594,0,2012-06-01,62421979000129


In [5]:
df_info_cadastral = G.table_to_df("finance_raw_data", "info_cadastral_entidades", spark)
df_info_cadastral.persist()

# dfSample_info_cadastral_entidades = df_info_cadastra l.sample(False, 0.1).limit(30)
# table = tabulate(dfSample_info_cadastral_entidades.collect(), headers=dfSample_info_cadastral_entidades.columns, tablefmt='html')
# display(HTML(table))

# df_info_cadastral.count()
# dfSample_info_cadastral_entidades.printSchema()


DataFrame[data: string, codigo_cadastro_bacen: string, codigo_sisbacen: string, codigo_pais_sede: string, nome_pais_sede: string, nome_uf_sede: string, codigo_municipio_sede: string, nome_municipio_sede: string, nome_entidade: string, nome_entidade_nao_formatado: string, cnpj: string, cnpj_raiz: string, codigo_situacao: string, descricao_situacao: string, codigo_tipo_entidade_segmento: string, nome_tipo_entidade: string, codigo_natureza_juridica: string, descricao_natureza_juridica: string, codigo_esfera_publica: string, nome_reduzido: string, sigla_entidade: string, nome_fantasia: string, empresa_publica: string]

In [6]:
df_info_cadastral_select_ = df_info_cadastral.select("cnpj", "data", "cnpj_raiz", "codigo_sisbacen", "nome_tipo_entidade", "descricao_situacao", "nome_entidade", "nome_fantasia")


df_info_cadastral_select = df_info_cadastral_select_.withColumn("data_format", F.to_date(df_info_cadastral_select_["data"], "yyyy-MM-dd"))
df_info_cadastral_select = df_info_cadastral_select.drop("data")
# a = df_info_cadastral_select.sample(False, 0.1).limit(50)
# table = tabulate(a.collect(), headers=a.columns, tablefmt='html')
# df_info_cadastral_select.filter(df_info_cadastral_select["cnpj"].isNull()).count()
# display(HTML(table))

df_info_cadastral_select.printSchema()


root
 |-- cnpj: string (nullable = true)
 |-- cnpj_raiz: string (nullable = true)
 |-- codigo_sisbacen: string (nullable = true)
 |-- nome_tipo_entidade: string (nullable = true)
 |-- descricao_situacao: string (nullable = true)
 |-- nome_entidade: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- data_format: date (nullable = true)



### Duplicação

In [7]:
# df_info_cadastral

df_info_ordered = df_info_cadastral_select.orderBy(F.col("cnpj").asc())

columns_info = df_info_ordered.columns
columns_info.remove("data_format")

df_unique_info_ = df_info_ordered.dropDuplicates(subset=columns_info)

# df_demonstracao_result

df_demonstracao_ordered = df_demonstracao_result.orderBy(F.col("cnpj_completo").asc())
columns_demonstracao = df_demonstracao_result.columns
columns_demonstracao.remove("data_trimestral")


df_unique_demonstracao_ = df_demonstracao_ordered.dropDuplicates(subset=columns_demonstracao)


# sample_info = df_unique_info_.sample(False, 0.1).limit(50)

# sample_info = df_unique_info_.filter(F.col("cnpj_raiz") == '00000000')

# tb_info = tabulate(sample_info.collect(), headers=sample_info.columns, tablefmt='html')
# display(HTML(tb_info))

# df1 = df_unique_info_.select("cnpj", "cnpj_raiz", "data_format").limit(30)
# df2 = df_unique_demonstracao_.select("instituicao", "cnpj_completo", "data_trimestral").limit(30)

df_unique_info_ = df_unique_info_.withColumn("ano_info", F.year("data_format")).withColumn("mes_info", F.month("data_format"))
df_unique_demonstracao_ = df_unique_demonstracao_.withColumn("ano", F.year("data_trimestral")).withColumn("mes", F.month("data_trimestral"))

df_unique_info_.printSchema()
df_unique_demonstracao_.printSchema()

# tb1 = tabulate(df1.collect(), headers=df1.columns, tablefmt='html')
# display(HTML(tb1))

# tb2 = tabulate(df2.collect(), headers=df2.columns, tablefmt='html')
# display(HTML(tb2))



root
 |-- cnpj: string (nullable = true)
 |-- cnpj_raiz: string (nullable = true)
 |-- codigo_sisbacen: string (nullable = true)
 |-- nome_tipo_entidade: string (nullable = true)
 |-- descricao_situacao: string (nullable = true)
 |-- nome_entidade: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- data_format: date (nullable = true)
 |-- ano_info: integer (nullable = true)
 |-- mes_info: integer (nullable = true)

root
 |-- instituicao: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- conglomerado: string (nullable = true)
 |-- codigo_conglomerado_financeiro: string (nullable = true)
 |-- codigo_conglomerado_prudencial: string (nullable = true)
 |-- tcb: string (nullable = true)
 |-- tc: string (nullable = true)
 |-- ti: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- rendas_operacoes_credito_a1: string (nullable = true)
 |-- rendas_operacoes_arrendamento_mercantil_a2: string (nullable = t

### Remover acentos

In [13]:
def clean_accent_(texto):
    return unidecode(texto) if texto else None

clean_accent = F.udf(clean_accent_, F.StringType())

col_info = ["nome_tipo_entidade", "descricao_situacao", "nome_entidade", "nome_fantasia"]
col_demonstracao = ["instituicao", "conglomerado", "tcb", "tc", "ti", "cidade", "uf"]

df_unique_info = df_unique_info_.select([F.upper(F.trim(clean_accent(F.col(column)))).alias(column) for column in col_info])

df_unique_demonstracao = df_unique_demonstracao_.select([F.upper(F.trim(clean_accent(F.col(column)))).alias(column) for column in col_demonstracao])

# df_unique_demonstracao.printSchema()

In [14]:
df_unique_info = df_unique_info_
df_unique_demonstracao = df_unique_demonstracao_


df_join_ = df_unique_info.join(
    df_unique_demonstracao,
    (df_unique_info["cnpj_raiz"] == df_unique_demonstracao["codigo"]) &
    (df_unique_info["ano_info"] == df_unique_demonstracao["ano"]) &
    (df_unique_info["mes_info"] == df_unique_demonstracao["mes"]),
    "full_outer"
).select(
    df_unique_demonstracao["instituicao"],
    df_unique_demonstracao["tc"],
    df_unique_demonstracao["conglomerado"],
    df_unique_demonstracao["cnpj_completo"],
    *df_unique_info.columns
)


df_join = df_join_.dropDuplicates()

# Filtrar registros que estão apenas no DataFrame "demonstracao_result"
only_demonstracao_result = df_join.filter(
    F.col("nome_entidade").isNull() &
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "")
)

# Filtrar registros que estão apenas no DataFrame "info_cadastral"
only_info_cadastral = df_join.filter(
    F.col("instituicao").isNull() &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)

# Filtrar registros que estão presentes em ambos DataFrames
intersecao = df_join.filter(
    F.col("instituicao").isNotNull() &
    (F.trim(F.col("instituicao")) != "") &
    F.col("nome_entidade").isNotNull() &
    (F.trim(F.col("nome_entidade")) != "")
)
print(only_demonstracao_result.count())

print(only_info_cadastral.count())

print(intersecao.count())

print("total", df_join.count())


# testee = df_sem_acento.filter(F.col("nome_entidade") == "COOPERATIVA DE CRÉDITO RURAL COM INTERAÇÃO SOLIDÁRIA DE CASTRO - CRESOL CASTRO")

# table = tabulate(testee.collect(), headers=testee.columns, tablefmt='html')
# display(HTML(table))



24/07/22 15:15:17 WARN TaskSetManager: Stage 12 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:18 WARN TaskSetManager: Stage 13 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:18 WARN TaskSetManager: Stage 14 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:20 WARN TaskSetManager: Stage 15 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


9653


24/07/22 15:15:27 WARN TaskSetManager: Stage 35 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:28 WARN TaskSetManager: Stage 36 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:28 WARN TaskSetManager: Stage 37 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:30 WARN TaskSetManager: Stage 38 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


5593


24/07/22 15:15:35 WARN TaskSetManager: Stage 46 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:37 WARN TaskSetManager: Stage 47 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:38 WARN TaskSetManager: Stage 48 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:39 WARN TaskSetManager: Stage 49 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.


0


24/07/22 15:15:43 WARN TaskSetManager: Stage 57 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:44 WARN TaskSetManager: Stage 58 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:44 WARN TaskSetManager: Stage 59 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/22 15:15:45 WARN TaskSetManager: Stage 60 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


total 15246


In [61]:
sample_only_demonstracao_result = only_demonstracao_result.sample(False, 0.1).limit(30)

tb_only_demonstracao_result = tabulate(only_demonstracao_result.collect(), headers=only_demonstracao_result.columns, tablefmt='html')

sample_only_info_cadastral = only_info_cadastral.sample(False, 0.1).limit(30)

tb_only_info_cadastral = tabulate(sample_only_info_cadastral.collect(), headers=sample_only_info_cadastral.columns, tablefmt='html')

sample_intersecao =intersecao.sample(False, 0.1).limit(30)

tb_intersecao = tabulate(sample_intersecao.collect(), headers=sample_intersecao.columns, tablefmt='html')

display(HTML(tb_only_demonstracao_result))
display(HTML(tb_only_info_cadastral))
display(HTML(tb_intersecao))



24/07/18 15:42:29 WARN TaskSetManager: Stage 371 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:29 WARN TaskSetManager: Stage 372 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:32 WARN TaskSetManager: Stage 377 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:33 WARN TaskSetManager: Stage 378 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:36 WARN TaskSetManager: Stage 383 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 15:42:37 WARN TaskSetManager: Stage 384 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
,,,,10664027000132,2024-05-23,10664027,,Sociedade Corretora de TVM,Cancelada/Encerrada,ABERTURA CORRETORA DE CAMBIO E VALORES MOBILIARIOS LTDA,
,,,,11547213000154,2024-05-23,11547213,,Administradora de Consórcio,Cancelada/Encerrada,ALMEIDA ADMINISTRACAO DE CONSORCIOS LTDA - IMPEDIDA CRIAR NOVOS GRUPOS,
,,,,33756370000102,2024-05-23,33756370,,Sociedade Corretora de TVM,Cancelada/Encerrada,"ARIJU S/A CORRETORA DE CAMBIO, TITULOS E VALORES MOBILIARIOS",
,,,,17338450000191,2024-05-23,17338450,,Sociedade Distribuidora de TVM,Regime Especial Encerrado,ASTRA DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA,ASTRA DIST. TITLS. VALS. MOBILIARIOS LTD
,,,,38726063000100,2024-05-23,38726063,,Administradora de Consórcio,Cancelada/Encerrada,AUTO & MOVEL CONSORCIOS LTDA,
,,,,15208309000149,2024-05-23,15208309,,"Sociedade de Crédito, Financiamento e Investimento",Cancelada/Encerrada,"BAHIA LUCRO S/A - CREDITO, FINANCIAMENTO E INVESTIMENTOS",
,,,,61415428000190,2024-05-23,61415428,,Banco Múltiplo,Cancelada/Encerrada,BANCO DE TOKYO S.A.,
,,,,45666609000132,2024-05-23,45666609,,Banco Comercial Estrangeiro - Filial no país,Cancelada/Encerrada,BANCO HISPANO AMERICANO S.A.,
,,,,60700556000465,2024-05-23,60700556,,Banco Múltiplo,Cancelada/Encerrada,BANCO SANTANDER NOROESTE S.A.,
,,,,74419714000175,2024-05-23,74419714,,Banco de Investimento,Cancelada/Encerrada,BANCO SINTESE DE INVESTIMENTO S.A.,


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
QUANTO NETWORK INSTITUICAO DE PAGAMENTO LTDA.,2,QUANTO NETWORK IP - PRUDENCIAL,42064545,42064545000101,2024-05-23,42064545,00005,Instituição de Pagamento,Autorizada em Atividade,QUANTO NETWORK INSTITUICAO DE PAGAMENTO LTDA.,QUANTO
BLU INSTITUIÇÃO DE PAGAMENTO LTDA.,2,BLU IP - PRUDENCIAL,10506341,10506341000197,2024-05-23,10506341,00059,Instituição de Pagamento,Autorizada em Atividade,BLU INSTITUIÇÃO DE PAGAMENTO LTDA.,BLU IP
ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,2,ZOOP TECNOLOGIA IP - PRUDENCIAL,19468242,19468242000213,2024-05-23,19468242,00137,Instituição de Pagamento,Autorizada em Atividade,ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,ZOOP TECNOLOGIA & INSTITUIÇÃO DE PAGAMENTO S.A.
BANCO BANERJ S.A.,2,ITAU,33885724,33885724000119,2024-05-23,33885724,04029,Banco Múltiplo,Autorizada em Atividade,BANCO ITAÚ CONSIGNADO S.A.,BANCO ITAÚ CONSIGNADO S.A.
BANCO ITAÚ BMG CONSIGNADO S.A.,2,ITAU,33885724,33885724000119,2024-05-23,33885724,04029,Banco Múltiplo,Autorizada em Atividade,BANCO ITAÚ CONSIGNADO S.A.,BANCO ITAÚ CONSIGNADO S.A.
UBS PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,3,UBS PACTUAL,43815158,43815158000122,2024-05-23,43815158,05211,Sociedade Corretora de TVM,Autorizada em Atividade,BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,BTG PACTUAL CTVM S.A.
BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,2,BTG PACTUAL,43815158,43815158000122,2024-05-23,43815158,05211,Sociedade Corretora de TVM,Autorizada em Atividade,BTG PACTUAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,BTG PACTUAL CTVM S.A.
BANCO GE CAPITAL S.A.,2,BMG,62421979,62421979000129,2024-05-23,62421979,05233,Banco Múltiplo,Autorizada em Atividade,BANCO CIFRA S.A.,BANCO CIFRA
BANCO LETSBANK S.A.,2,BANCO VOITER - PRUDENCIAL,58497702,58497702000102,2024-05-23,58497702,05630,Banco Múltiplo,Autorizada em Atividade,BANCO LETSBANK S.A.,LETSBANK
BANCO ITAUBANK S.A.,2,ITAU - PRUDENCIAL,60394079,60394079000104,2024-05-23,60394079,06479,Banco Múltiplo,Autorizada em Atividade,BANCO ITAUBANK S.A.,ITAUBANK


In [13]:
df_join.printSchema()
df_join_sample = df_join.sample(False, 0.1).limit(30)

tb = tabulate(df_join_sample.collect(), headers=df_join_sample.columns, tablefmt='html')

display(HTML(tb))

root
 |-- instituicao: string (nullable = true)
 |-- tc: string (nullable = true)
 |-- conglomerado: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- data: string (nullable = true)
 |-- cnpj_raiz: string (nullable = true)
 |-- codigo_sisbacen: string (nullable = true)
 |-- nome_tipo_entidade: string (nullable = true)
 |-- descricao_situacao: string (nullable = true)
 |-- nome_entidade: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)



24/07/18 17:15:04 WARN TaskSetManager: Stage 110 contains a task of very large size (12111 KiB). The maximum recommended task size is 1000 KiB.
24/07/18 17:15:04 WARN TaskSetManager: Stage 111 contains a task of very large size (11648 KiB). The maximum recommended task size is 1000 KiB.


instituicao,tc,conglomerado,codigo,cnpj,data,cnpj_raiz,codigo_sisbacen,nome_tipo_entidade,descricao_situacao,nome_entidade,nome_fantasia
CATERPILLAR FINANCIAL S.A. ARRENDAMENTO MERCANTIL,3,CATERPILLAR,02658435,02658435000153,2024-05-23,02658435,53154,Banco Múltiplo,Autorizada em Atividade,BANCO CATERPILLAR S.A.,BANCO CATERPILLAR S.A.
BANCO PRIME DE INVESTIMENTO S.A.,2,PRIME,45246410,45246410000155,2024-05-23,45246410,31710,Banco Múltiplo,Autorizada em Atividade,BANCO GENIAL S.A.,PLURAL BANCO
CM CAPITAL MARKETS CORRETORA DE CAMBIO LTDA,3,CM CAPITAL MARKETS,02671743,02671743000119,2024-06-12,02671743,57538,Sociedade Distribuidora de TVM,Autorizada em Atividade,BANVOX DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA,BANVOX DTVM
BANCO FINAXIS S.A.,2,FINAXIS - PRUDENCIAL,11758741,11758741000152,2024-05-23,11758741,27731,Banco Comercial,Autorizada em Atividade,BANCO FINAXIS S.A.,BANCO FINAXIS S.A.
BANCO INDUSVAL S.A.,2,INDUSVAL,61024352,61024352000171,2024-05-23,61024352,05653,Banco Múltiplo,Autorizada em Atividade,BANCO VOITER S.A.,VOITER
BANKPAR BANCO MÚLTIPLO S.A.,2,BRADESCO,59438325,59438325000101,2024-05-23,59438325,07204,Banco Múltiplo,Cancelada/Encerrada,BANCO BRADESCO CARTÕES S.A.,BRADESCO CARTÕES
STONE SOCIEDADE DE CRÉDITO DIRETO S.A.,2,STONE PAGAMENTOS - PRUDENCIAL,34590184,34590184000109,2024-05-23,34590184,42504,Sociedade de Crédito Direto,Autorizada em Atividade,STONE SOCIEDADE DE CRÉDITO DIRETO S.A.,STONE SCD
SAFRA DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA,2,SAFRA,62180047,62180047000131,2024-05-23,62180047,37383,Sociedade Distribuidora de TVM,Cancelada/Encerrada,SAFRA ASSET MANAGEMENT LTDA,SAFRA ASSET MANAGEMENT
M18 INSTITUIÇÃO DE PAGAMENTO S.A.,2,M18 IP - PRUDENCIAL,22575466,22575466000158,2024-05-23,22575466,41921,Instituição de Pagamento,Autorizada em Atividade,SRM BANK INSTITUIÇÃO DE PAGAMENTO S/A,SRM BANK
SANTANDER SECURITIES (BRASIL)CORRETORA DE VALORES MOBILIÁRIOS S.A.,3,SANTANDER,61733515,61733515000195,2024-05-23,61733515,57054,Sociedade Corretora de TVM,Cancelada/Encerrada,SANTANDER SECURITIES (BRASIL)CORRETORA DE VALORES MOBILIÁRIOS S.A.,SANTANDER SECURITIES CVM


['codigo_cadastro_bacen', 'codigo_sisbacen', 'codigo_pais_sede', 'nome_pais_sede', 'nome_uf_sede', 'codigo_municipio_sede', 'nome_municipio_sede', 'nome_entidade', 'nome_entidade_nao_formatado', 'cnpj', 'cnpj_raiz', 'codigo_situacao', 'descricao_situacao', 'codigo_tipo_entidade_segmento', 'nome_tipo_entidade', 'codigo_natureza_juridica', 'descricao_natureza_juridica', 'codigo_esfera_publica', 'nome_reduzido', 'sigla_entidade', 'nome_fantasia', 'empresa_publica']
<class 'list'>


In [58]:
data = [("1",), ("123",), ("12345",), ("12345678",)]
columns = ["codigo"]
df = spark.createDataFrame(data, columns)

# Adicionar zeros à esquerda para que todos os valores na coluna "codigo" tenham exatamente 8 dígitos
df_formatado = df.withColumn("codigo", F.lpad(df["codigo"], 8, '0'))

# Exibir o DataFrame formatado
df_formatado.show(truncate=False)

+--------+
|codigo  |
+--------+
|00000001|
|00000123|
|00012345|
|12345678|
+--------+

